# Pokedex

imports

In [26]:
import os
import cv2
import glob
import sklearn
import numpy
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split
import random
from sklearn.ensemble import RandomForestClassifier

parametros globais

In [31]:
#parametros globais
raiz_dataSet = './images/images/'

#obtem as subpastas dentro de uma pasta
def get_folders(data_base):
  data_folders = []
  for name in os.listdir(data_base):
    if(os.path.isdir(data_base + name)):
      data_folders.append(name)
  #print(data_folders)
  return data_folders

Pre processamentos

In [11]:

def resize(im, par):
  im = cv2.resize(im, (par[0],par[0]))
  return im


def gaussian(im, par):
  im = cv2.GaussianBlur(im, (par[0],par[0]), par[1])
  return im

def median(im, par):
  im = cv2.medianBlur(im,par[0])
  return im


def bilateral(im, par):
  im = cv2.bilateralFilter(im,par[0],par[0]*1,par[0]/2)
  return im


def gamma(im, par):
	invGamma = 1.0 / par[0]
	table = numpy.array([((i / 255.0) ** invGamma) * 255
		for i in numpy.arange(0, 256)]).astype("uint8")
	return cv2.LUT(im, table)


def equalize(im,par):
  return cv2.equalizeHist(im)

'''
'log' : {
      'value' : [0]
    }
'''
def logfilter(im, par):
  m = numpy.max(im)
  if m == 0:
    m = 255
  c = 255 / numpy.log(1 + m)
  log_image = c * (numpy.log(im + 1))
  log_image = numpy.array(log_image, dtype = numpy.uint8)
  return log_image

'''
'clahe' : {
      'clipLimit' : [2, 4, 6],
      'size' : [8, 12, 16]
    }
'''
def clahe(im, par):
  clahe = cv2.createCLAHE(clipLimit=par[0], tileGridSize=(par[1],par[1]))
  cl1 = clahe.apply(im)
  return cl1

Otimização

In [12]:
#um hog básico
def compute_hog(im):
  winSize = im.shape
  blockSize = (int(winSize[0]/blocks), int(winSize[1]/blocks))
  blockStride = blockSize
  cellSize = (int(blockSize[0]/2), int(blockSize[1]/2))
  hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)
  vetor = hog.compute(im)
  return numpy.reshape(vetor, (vetor.shape[0]))

#parâmetros HOG
blocks = 4
nbins = 12


def extract_features():

  #retorne X, y
  return
#otimize

In [ ]:
#defina as funções de otimização para serem avaliadas
parametros = {
    'clahe' : {
      'clipLimit' : [4, 2],
      'size' : [16, 12, 8]
    },
    'log' : {
      'value' : [0]
    },
    'equalize' : {
      'value' : [0]
    },
    'gamma' : {
        'value' : [0.5, 1, 1.5, 2]
    },
    'bilateral' : {
        'size' : [3, 5, 7, 9, 11, 13]
    },
    'median' : {
        'filter' : [3, 5, 7]
    },
    'resize' : {
        'size' : [128, 256, 512]
    },
    'gaussian' : {
        'filter' : [3, 5, 7],
        'desv'   : [0, 0.5, 1, 1.5]
    }


}

#quando se deseja encontrar apenas uma função de pré-processamento
def evaluateGridOne(parametros):
  #print('Labels: ')
  #carrega a base

  df = pd.read_csv(raiz_dataSet+'pokemon.csv',index_col='Name')
  df = df.drop(['Type2'], axis=1)
  best_size = 128 #vai ser usado como padrão depois para os demais métodos

  best_score = 0
  best_param = []


  #1: para cada algoritmos de pre-processamento
  for alg, pars in parametros.items():
    print("Alg:" + alg, end=" ")
    items = list(product(*pars.values()))
    #2: para cada parâmetro
    for par in items:
      print("Par:" + str(par), end=" ")
      data = []
      labels = []
      dataset = glob.glob(raiz_dataSet + "/*" )#+ image_format
      print(dataset)
      for arq in dataset:
        print(arq)
        #pega a imagem
        im = cv2.imread(arq,0)
        #3: pre processa
        if alg == 'resize':
          im = resize(im, par)
        elif alg == 'gaussian':
          im = cv2.resize(im, (best_size, best_size))
          im = gaussian(im, par)
        elif alg == 'median':
          im = cv2.resize(im, (best_size, best_size))
          im = median(im, par)
        elif alg == 'bilateral':
          im = cv2.resize(im, (best_size, best_size))
          im = bilateral(im, par)
        elif alg == 'gamma':
          im = cv2.resize(im, (best_size, best_size))
          im = gamma(im, par)
        elif alg == 'equalize':
          im = cv2.resize(im, (best_size, best_size))
          im = equalize(im, par)
        elif alg == 'log':
          im = cv2.resize(im, (best_size, best_size))
          im = logfilter(im, par)
        elif alg == 'clahe':
          im = cv2.resize(im, (best_size, best_size))
          im = clahe(im, par)

        #computa as características
        data.append(compute_hog(im))
        nome = arq
        nome = nome.split('.')
        print(nome)
        nome[0]
        print(nome[0])
        #adiciona as labels
        labels.append(df.loc[nome[0]]['Type1'])

      #conjunto completo
      X = numpy.asarray(data)
      y = numpy.asarray(labels)


      #4: estima o resultado
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.3) #sempre o mesmo aleatório para dar sempre a mesma base

      #o classificador é apenas para realizar um fitness do problema
      clf = RandomForestClassifier(n_estimators=100, max_depth=3,random_state=random.randrange(1, 1000)).fit(X_train, y_train)
      y_get = clf.predict(X_test)
      score = sklearn.metrics.balanced_accuracy_score(y_test, y_get)
      print("Balanced Acc: " + str(score), end="\n")
      if score > best_score:
        best_score = score
        best_param = par
        if alg == 'resize':
          best_size = par[0]

    print("Best param alg: " + alg + ": " + str(best_param))
    best_param = []
    best_score = 0

evaluateGridOne(parametros)

Hog Basico